In [15]:
import sys
sys.path.append('./code')
import argparse
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import argparse
from vgg_face import *
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
plt.switch_backend('agg')


In [16]:
def set_parameter_requires_grad(model, feature_extracting):
    for param in model.parameters():
        param.requires_grad = False if feature_extracting else True

def centercrop_opencv(image, size):
    height, width, _ = image.shape
    crop_height = crop_width = size
    start_height = (height - crop_height) // 2
    start_width = (width - crop_width) // 2
    cropped_image = image[start_height:start_height+crop_height, start_width:start_width+crop_width]
    return cropped_image

def test_model(model, crop_frame, device):
    model.eval()
    with torch.no_grad():
        crop_frame = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2RGB)
        crop_frame = cv2.resize(crop_frame, (256, 256))
        crop_frame = centercrop_opencv(crop_frame, 224)
        crop_frame = crop_frame / 255.0
        crop_frame = np.transpose(crop_frame, (2, 0, 1))
        crop_frame = np.expand_dims(crop_frame, axis=0)  # add batch dimension
        crop_frame = torch.from_numpy(crop_frame).float().to(device)
        output = model(crop_frame)
        output = output * torch.FloatTensor([16] + [5]*9 + [25]).to(device)
    return output

def plot_bar(au_scores, title):
    #labels = ['AU4', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU20', 'AU25', 'AU26', 'AU43']
    labels = ['Brow\nLowerer', 'Cheek\nRaiser', 'Lid\nTightener', 'Nose\nWrinkler', 'Upper Lip\nRaiser', 'Lip Corner\nPuller', 'Lip\nStretcher', 'Lips\nPart', 'Jaw\nDrop', 'Eyes\nClosed']
    colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'burlywood']
    fig, ax = plt.subplots(figsize=(10, 10.8))
    values = au_scores[1:]
    fig, ax = plt.subplots(figsize=(10, 10.8))

    #ax.barh(labels, values, color=colors)
    ordertoshow=[7, 6, 5, 4, 3, 2, 8, 0, 1, 9]
    ax.barh(np.take(labels, ordertoshow), np.take(values, ordertoshow), color=np.take(colors, ordertoshow))

    ax.tick_params(axis='both', labelsize=15)
    ax.set_title(title + ' PSPI {:.2f}/16'.format(au_scores[0]), fontsize=20)
    ax.set_xlim([0, 5])
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    plot = np.array(canvas.renderer.buffer_rgba())
    plot = cv2.cvtColor(plot, cv2.COLOR_RGB2BGR)
    plt.close()
    return plot

def rounded_rectangle(image, x, y, w, h, radius, thickness):
    cv2.ellipse(image, (x + radius, y + radius), (radius, radius), 180, 0, 90, (51,255,255), thickness)
    cv2.ellipse(image, (x + w - radius, y + radius), (radius, radius), 270, 0, 90, (51,255,255), thickness)
    cv2.ellipse(image, (x + radius, y + h - radius), (radius, radius), 90, 0, 90, (51,255,255), thickness)
    cv2.ellipse(image, (x + w - radius, y + h - radius), (radius, radius), 0, 0, 90, (51,255,255), thickness)
    return image

def adaptive_canvas(image_left, image_right):
    h_l, w_l, _ = image_left.shape
    h_r, w_r, _ = image_right.shape
    canvas_h = max(h_l, h_r)
    canvas_w = w_l+w_r
    canvas = np.zeros((canvas_h, canvas_w, 3), dtype=np.uint8)
    canvas[:h_l, :w_l, :] = image_left[:,:,:3]
    canvas[:h_r, w_l:, :] = image_right[:,:,:3]
    return canvas

In [17]:
num_classes = 11
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = VGG_16().to(device)
set_parameter_requires_grad(model, feature_extracting=False)
num_ftrs = model.fc8.in_features
model.fc8 = nn.Linear(num_ftrs, num_classes).to(device)
model.load_state_dict(torch.load('/Users/miguelmonares/Downloads/drive-download-20230524T190919Z-001/checkpoint_epoch69_11.pth', map_location=device))
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt_tree.xml')

In [18]:
class args:
    scale = 80
    round = 1
    square = 0
    sampling_rate = 1

In [20]:
frame = cv2.imread('/Users/miguelmonares/FAU/images/European_Man/em1/4em1_4.10.png')

frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(frame_gray, 1.1, 6)
for (x, y, w, h) in faces:
    h = w
    crop_frame = np.copy(frame_flip)[y-2*args.scale:y+h+args.scale, x-args.scale:x+w+args.scale]
    if args.round == 1:
        frame_flip = rounded_rectangle(frame_flip, x, y, w, h, 100, 20)
    if args.square == 1:
        frame_flip = cv2.rectangle(frame_flip, (x-args.scale, y-2*args.scale), (x+w+args.scale, y+h+args.scale), (255, 255, 255), 4)

cv2.imshow('img',crop_frame)

NameError: name 'crop_frame' is not defined

In [19]:
cv2.imshow('img',crop_frame)

NameError: name 'crop_frame' is not defined

In [5]:
# set up a camera window
cv2.namedWindow('face_cap', cv2.WINDOW_NORMAL)
cv2.resizeWindow('face_cap', 1500, 600)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
cap_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
cap_info = '({}*{}, fps {})'.format(cap_w, cap_h, fps)

save_count=0
while(True):
    ret, frame = cap.read()

    # mirror the video
    frame_flip = cv2.flip(frame,1)

    # opencv2 detect face
    frame_gray = cv2.cvtColor(frame_flip, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame_gray, 1.1, 6)
    for (x, y, w, h) in faces:
        print("A")
        h = w
        crop_frame = np.copy(frame_flip)[y-2*args.scale:y+h+args.scale, x-args.scale:x+w+args.scale]
        if args.round == 1:
            frame_flip = rounded_rectangle(frame_flip, x, y, w, h, 100, 20)
        if args.square == 1:
            frame_flip = cv2.rectangle(frame_flip, (x-args.scale, y-2*args.scale), (x+w+args.scale, y+h+args.scale), (255, 255, 255), 4)

    # feed the frame into the model
    try:
        output = test_model(model, crop_frame, device)
        output = output.flatten()
        output_cpu = torch.Tensor.cpu(output).numpy()
        output_plot = plot_bar(output_cpu, cap_info)
        output_text = '|PSPI {:.2f} |au4 {:.2f} |au6 {:.2f} |au7 {:.2f} |au9 {:.2f} |au10 {:.2f} |au12 {:.2f} |au20 {:.2f} |au25 {:.2f} |au26 {:.2f} |au43 {:.2f}|'.format(output[0].item(), output[1].item(), output[2].item(), output[3].item(), output[4].item(), output[5].item(), output[6].item(), output[7].item(), output[8].item(), output[9].item(), output[10].item())
        print(output_text)

        # save video option


    except Exception as error:
        output_plot = canvas = np.ones((1080, 1000, 3), dtype=np.uint8)*255
        output_text = str(error)

    # print AU info at the bottom of each frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    #thickness = 2
    text_size, _ = cv2.getTextSize(output_text, font, font_scale, 2)
    text_x = int((cap_w - text_size[0]) / 2)
    text_y = cap_h - text_size[1] - 10
    cv2.putText(frame_flip, output_text, (text_x, text_y), font, font_scale, (255, 255, 255), 2)

    # dispay canvas
    canvas = adaptive_canvas(frame_flip, output_plot)
    plt.close()
    cv2.imshow('face_cap', canvas)

    # Press 'Q' to quit camera
    if cv2.waitKey(1) == ord('q'):
        break

    # After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [11]:
import os
import cv2

def crop_images(input_folder, output_folder, x, y, width, height):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    file_list = os.listdir(input_folder)

    # Loop through each file in the input folder
    for file_name in file_list:
        # Check if the file is an image
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Read the image
            image_path = os.path.join(input_folder, file_name)
            image = cv2.imread(image_path)

            # Crop the image
            cropped_image = image[y:y+height, x:x+width]

            # Save the cropped image
            output_path = os.path.join(output_folder, file_name)
            cv2.imwrite(output_path, cropped_image)

            print(f"Cropped image saved: {output_path}")

# Example usage
input_folder = "/Users/miguelmonares/FAU/images/European_Woman/ew9"
output_folder = "/Users/miguelmonares/FAU2/images/European_Woman/ew9"
x = 880  # x-coordinate of the top-left corner of the crop rectangle
y = 180  # y-coordinate of the top-left corner of the crop rectangle
width = 750  # Width of the crop rectangle
height = 820  # Height of the crop rectangle

crop_images(input_folder, output_folder, x, y, width, height)


Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/7ew9_7.10.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/7ew9_7.4.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/6ew9_6.8.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/9ew9_9.6.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/25ew9_25.4.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/25ew9_25.6.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/9ew9_9.4.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/10ew9_10.2.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/7ew9_7.6.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/26ew9_26.2.png
Cropped image saved: /Users/miguelmonares/FAU2/images/European_Woman/ew9/12ew9_12.8.png
Cropped image saved: /Users/miguelmonares/F